In [10]:
import json, unidecode, re
from pathlib import Path
from pprint import pprint

data = Path('data')

# Load the json file
with (data / 'animes.json').open() as f:
	animes = json.load(f)

pprint(animes[0])

{'aired': {'from': '2023-04-12T00:00:00+00:00',
           'prop': {'from': {'day': 12, 'month': 4, 'year': 2023},
                    'to': {'day': 28, 'month': 6, 'year': 2023}},
           'string': 'Apr 12, 2023 to Jun 28, 2023',
           'to': '2023-06-28T00:00:00+00:00'},
 'airing': False,
 'approved': True,
 'background': None,
 'broadcast': {'day': 'Wednesdays',
               'string': 'Wednesdays at 23:00 (JST)',
               'time': '23:00',
               'timezone': 'Asia/Tokyo'},
 'demographics': [{'mal_id': 42,
                   'name': 'Seinen',
                   'type': 'anime',
                   'url': 'https://myanimelist.net/anime/genre/42/Seinen'}],
 'duration': '30 min per ep',
 'episodes': 11,
 'explicit_genres': [],
 'favorites': 23559,
 'genres': [{'mal_id': 8,
             'name': 'Drama',
             'type': 'anime',
             'url': 'https://myanimelist.net/anime/genre/8/Drama'},
            {'mal_id': 37,
             'name': 'Supernatural',
    

# Aggregate animes into franchises

In [11]:
def sanitize(text: str):
	# Unidecode
	text = unidecode.unidecode(text)
	# Lowercase
	text = text.lower()
	# Remove special characters
	text = re.sub(r"[^\w\s]", "", text)
	# Remove multiple spaces
	text = re.sub(r"\s+", " ", text)
	# Remove leading and trailing spaces
	text = text.strip()
	return text

# Return the name of the franchise, none if not found
def get_franchise(a_title: str, f_title: str, auto: bool):
	words1 = a_title.split(" ")
	words2 = f_title.split(" ")
	min_w_len = min(len(words1), len(words2))

	# If franchise isn't auto, and we find a common string, it is a franchise
	match = re.search(f_title, a_title)
	if not auto and match:
		# print(f"Franchise match (manual)")
		return f_title

	common = []
	for i in range(min_w_len):
		if sanitize(words1[i]) == sanitize(words2[i]):
			common.append(words1[i])
		else:
			break
	franchise = " ".join(common)

	# If more than XX% of the characters of the shortest title are common, it is a franchise
	min_len = min(len(a_title), len(f_title))
	if len(franchise) / min_len > 0.8:
		# print(f"Franchise match (XX%): {len(franchise)}/{min_len}")
		return franchise

	# If the length of the common string is more than X characters, it is a franchise
	if len(franchise) > 15:
		# print(f"Franchise match (X characters)")
		return franchise

	return None

known_franchises = [
	"Evangelion",
	"Code Geass",
]

franchises = []
# Initialize franchises list with known franchises
for franchise in known_franchises:
	franchises.append({
		"animes": [],
		"title": franchise,
		"auto": False,
	})

# Build franchises list
for anime in animes:
	match = None
	index = None

	for i in range(len(franchises)):
		franchise = franchises[i]
		match = get_franchise(anime["title"], franchise["title"], franchise["auto"])
		if match:
			# print(f"Franchise of [{anime['title']}] and [{franchises[i]['title']}] is [{match}]")
			index = i
			break
	
	if not match:
		franchises.append({
			"animes": [anime],
			"title": anime["title"],
			"auto": True,
		})
		continue

	franchises[index]["animes"].append(anime)
	franchises[index]["title"] = match

print("Found " + str(len(franchises)) + " franchises")
for franchise in franchises:
	titles = [anime["title"] for anime in franchise["animes"]]
	print(f"- {franchise['title']} [{', '.join(titles)}]")

Found 105 franchises
- Evangelion [Evangelion: 1.0 You Are (Not) Alone, Evangelion: 2.0 You Can (Not) Advance, Evangelion: 3.0 You Can (Not) Redo, Evangelion: 3.0+1.0 Thrice Upon a Time, Neon Genesis Evangelion]
- Code Geass [Code Geass: Fukkatsu no Lelouch, Code Geass: Hangyaku no Lelouch, Code Geass: Hangyaku no Lelouch R2]
- "Oshi no Ko" ["Oshi no Ko", "Oshi no Ko" Season 2]
- 3-gatsu no Lion [3-gatsu no Lion]
- 86 [86, 86 Part 2]
- Angel Beats! [Angel Beats!]
- Ansatsu Kyoushitsu [Ansatsu Kyoushitsu, Ansatsu Kyoushitsu 2nd Season]
- Azumanga Daiou The Animation [Azumanga Daiou The Animation]
- Banana Fish [Banana Fish]
- Beastars [Beastars]
- Berserk: Ougon Jidai-hen [Berserk: Ougon Jidai-hen I - Haou no Tamago, Berserk: Ougon Jidai-hen II - Doldrey Kouryaku, Berserk: Ougon Jidai-hen III - Kourin]
- Bleach: Sennen Kessen-hen [Bleach: Sennen Kessen-hen]
- Blue Giant [Blue Giant]
- Blue Lock [Blue Lock, Blue Lock 2nd Season]
- Bocchi the Rock! [Bocchi the Rock!, Bocchi the Rock! Movi

In [12]:
# Save the franchises
with (data / 'franchises.json').open('w') as f:
	json.dump(franchises, f)

# Compute franchise stats

In [13]:
def weighted_mean(animes, attr, total_episodes):
    if total_episodes > 0:  # Check to avoid division by zero
        return sum((anime[attr] if anime[attr] is not None else 0) * 
                   (anime["episodes"] if anime["episodes"] is not None else 0) 
                   for anime in animes) / total_episodes
    return None

def union(animes, attr):
    return list(set(name['name'] for anime in animes for name in anime[attr]))

for franchise in franchises:
    animes = franchise['animes']
    franchise['episodes'] = sum(anime["episodes"] if anime["episodes"] is not None else 0 for anime in animes)
    franchise['score'] = weighted_mean(animes, "score", franchise['episodes'])
    franchise['my_score'] = weighted_mean(animes, "my_score", franchise['episodes'])
    franchise['genres'] = union(animes, "genres")
    franchise['themes'] = union(animes, "themes")
    franchise['demographics'] = union(animes, "demographics")

franchises[0]  # Display the first franchise to check the results

{'animes': [{'mal_id': 2759,
   'url': 'https://myanimelist.net/anime/2759/Evangelion__10_You_Are_Not_Alone',
   'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/anime/7/74975.jpg',
     'small_image_url': 'https://cdn.myanimelist.net/images/anime/7/74975t.jpg',
     'large_image_url': 'https://cdn.myanimelist.net/images/anime/7/74975l.jpg'},
    'webp': {'image_url': 'https://cdn.myanimelist.net/images/anime/7/74975.webp',
     'small_image_url': 'https://cdn.myanimelist.net/images/anime/7/74975t.webp',
     'large_image_url': 'https://cdn.myanimelist.net/images/anime/7/74975l.webp'}},
   'trailer': {'youtube_id': 'ETNj92NPIh4',
    'url': 'https://www.youtube.com/watch?v=ETNj92NPIh4',
    'embed_url': 'https://www.youtube.com/embed/ETNj92NPIh4?enablejsapi=1&wmode=opaque&autoplay=1',
    'images': {'image_url': 'https://img.youtube.com/vi/ETNj92NPIh4/default.jpg',
     'small_image_url': 'https://img.youtube.com/vi/ETNj92NPIh4/sddefault.jpg',
     'medium_image_url'